# PyTorch Assignment: Multi-Layer Perceptron (MLP)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Natalie Chai

### Multi-Layer Perceptrons

The simple logistic regression example we went over in the previous notebook is essentially a one-layer neural network, projecting straight from the input to the output predictions.
While this can be effective for linearly separable data, occasionally a little more complexity is necessary.
Neural networks with additional layers are typically able to learn more complex functions, leading to better performance.
These additional layers (called "hidden" layers) transform the input into one or more intermediate representations before making a final prediction.

In the logistic regression example, the way we performed the transformation was with a fully-connected layer, which consisted of a linear transform (matrix multiply plus a bias).
A neural network consisting of multiple successive fully-connected layers is commonly called a Multi-Layer Perceptron (MLP). 
In the simple MLP below, a 4-d input is projected to a 5-d hidden representation, which is then projected to a single output that is used to make the final prediction.

<center>
    <img src="Figures/MLP.png" width="300"/>
</center>

For the assignment, you will be building a MLP for MNIST.
Mechanically, this is done very similary to our logistic regression example, but instead of going straight to a 10-d vector representing our output predictions, we might first transform to a 500-d vector with a "hidden" layer, then to the output of dimension 10.
Before you do so, however, there's one more important thing to consider.

### Nonlinearities

We typically include nonlinearities between layers of a neural network.
There's a number of reasons to do so.
For one, without anything nonlinear between them, successive linear transforms (fully connected layers) collapse into a single linear transform, which means the model isn't any more expressive than a single layer.
On the other hand, intermediate nonlinearities prevent this collapse, allowing neural networks to approximate more complex functions.

There are a number of nonlinearities commonly used in neural networks, but one of the most popular is the [rectified linear unit (ReLU)](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)):

\begin{align}
x = \max(0,x)
\end{align}

There are a number of ways to implement this in PyTorch.
We could do it with elementary PyTorch operations:

In [1]:
import torch

x = torch.rand(5, 3)*2 - 1
x_relu_max = torch.max(torch.zeros_like(x),x)

print("x: {}".format(x))
print("x after ReLU with max: {}".format(x_relu_max))

x: tensor([[ 0.1118,  0.5887,  0.6623],
        [-0.7509,  0.1526, -0.9066],
        [-0.6129, -0.5254, -0.1173],
        [-0.2582,  0.4956, -0.3517],
        [-0.4038, -0.6516, -0.1544]])
x after ReLU with max: tensor([[0.1118, 0.5887, 0.6623],
        [0.0000, 0.1526, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.4956, 0.0000],
        [0.0000, 0.0000, 0.0000]])


Of course, PyTorch also has the ReLU implemented, for example in `torch.nn.functional`:

In [2]:
import torch.nn.functional as F

x_relu_F = F.relu(x)

print("x after ReLU with nn.functional: {}".format(x_relu_F))

x after ReLU with nn.functional: tensor([[0.1118, 0.5887, 0.6623],
        [0.0000, 0.1526, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.4956, 0.0000],
        [0.0000, 0.0000, 0.0000]])


### Assignment

Build a 2-layer MLP for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image (784 dimensions) ->  
fully connected layer (500 hidden units) -> nonlinearity (ReLU) ->  
fully connected (10 hidden units) -> softmax

Try building the model both with basic PyTorch operations, and then again with more object-oriented higher-level APIs. 
You should get similar results!


*Some hints*:
- Even as we add additional layers, we still only require a single optimizer to learn the parameters.
Just make sure to pass all parameters to it!
- As you'll calculate in the Short Answer, this MLP model has many more parameters than the logisitic regression example, which makes it more challenging to learn.
To get the best performance, you may want to play with the learning rate and increase the number of training epochs.
- Be careful using `torch.nn.CrossEntropyLoss()`. 
If you look at the [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#crossentropyloss): you'll see that `torch.nn.CrossEntropyLoss()` combines the softmax operation with the cross-entropy.
This means you need to pass in the logits (predictions pre-softmax) to this loss.
Computing the softmax separately and feeding the result into `torch.nn.CrossEntropyLoss()` will significantly degrade your model's performance!

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

# Load the data
mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)

# Common Training Parameters 
num_epochs = 10 # Increased epochs for better performance on MLP
learning_rate = 0.2 # Adjusted learning rate, feel free to experiment

class MNIST_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Using Kaiming Uniform initialization heuristic (approx. sqrt(2/fan_in)) for ReLU
        self.W1 = nn.Parameter(torch.randn(784, 500) * np.sqrt(2./784))
        self.b1 = nn.Parameter(torch.zeros(500))

        self.W2 = nn.Parameter(torch.randn(500, 10) * np.sqrt(2./500))
        self.b2 = nn.Parameter(torch.zeros(10))

    def forward(self, x):
        # Reshape input to (batch_size, 784)
        x = x.view(-1, 28*28)

        # First fully connected layer + ReLU activation
        # Manual matrix multiplication: input @ weight + bias
        hidden_output = x @ self.W1 + self.b1
        hidden_output_relu = F.relu(hidden_output) # ReLU non-linearity

        # Second fully connected layer (output logits)
        logits = hidden_output_relu @ self.W2 + self.b2
        return logits

## Training
model = MNIST_MLP()
criterion = nn.CrossEntropyLoss() # Combines LogSoftmax and NLLLoss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Iterate through epochs
for epoch in range(num_epochs):
    model.train() # Set model to training mode
    running_loss = 0.0
    # Iterate through train set batches
    for images, labels in tqdm(train_loader, desc=f"Manual Epoch {epoch+1}/{num_epochs}"):
        # Zero out gradients
        optimizer.zero_grad() 
        
        # Forward pass
        y_pred = model(images) 
        loss = criterion(y_pred, labels) # Calculate loss
        
        # Backward pass (compute gradients)
        loss.backward() 
        optimizer.step() # Update weights
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Manual Epoch {epoch+1} Loss: {epoch_loss:.4f}")

## Testing
model.eval() # Set model to evaluation mode
correct = 0
total = len(mnist_test)

with torch.no_grad(): # Disable gradient calculation for inference
    # Iterate through test set batches
    for images, labels in tqdm(test_loader):
        x = images.view(-1, 28*28) # Reshape input
        y_pred = model(x) # Forward pass
        
        predictions = torch.argmax(y_pred, dim=1) # Get predicted class
        correct += torch.sum((predictions == labels).float()) # Count correct predictions

print('MLP Test Accuracy: {:.4f}'.format(correct/total))

Manual Epoch 1/10:   0%|          | 0/600 [00:00<?, ?it/s]

Manual Epoch 1 Loss: 0.3257


Manual Epoch 2/10:   0%|          | 0/600 [00:00<?, ?it/s]

Manual Epoch 2 Loss: 0.1604


Manual Epoch 3/10:   0%|          | 0/600 [00:00<?, ?it/s]

...

Manual Epoch 9 Loss: 0.0393


Manual Epoch 10/10:   0%|          | 0/600 [00:00<?, ?it/s]

Manual Epoch 10 Loss: 0.0340


  0%|          | 0/100 [00:00<?, ?it/s]

MLP Test Accuracy: 0.9796


### Short answer
How many trainable parameters does your model have? 
How does this compare to the logisitic regression example?

`2. MLP performs better as compared to logistic regression, with lower lost and highest performance.`